In [1]:
!pip3 install wget
!pip3 install jupyter-dash

In [2]:
import wget
import pandas as pd

In [3]:
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import dash
import plotly.graph_objects as go
import plotly.express as px
import multiprocessing
import jupyterlab_dash
from jupyter_dash import JupyterDash

# Read the airline data into pandas dataframe
spacex_df=pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
site = spacex_df['Launch Site'].unique()

# Create a dash application
app = JupyterDash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                             options = [{'label': 'All Sites', 'value': 'ALL'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}],
                                             value = 'ALL',
                                             placeholder='Select Launch Site',
                                             searchable=True),
                                html.Br(),
                                # TASK: Add a pie chart to show the total successful
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                               

                                # TASK 3: Add a slider to select payload range
                                html.P("Payload Range (KG)"),
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0 Kg',
                                                    2500: '2500',
                                                    5000: '5000',
                                                    7500: '7500',
                                                    10000: '10000'},
                                                value=[min_payload, max_payload]
                                ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),                           
                                ])

# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(site):
    filtered_df=spacex_df
    if site == 'ALL':
        fig = px.pie(spacex_df, values = 'class', names='Launch Site', title="Success Count for All Launch Sites")
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig2 = px.pie(filtered_df, values = 'class count', names = 'class', title=f"Total Success Launches for {site}")
        return fig2
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])

def get_scatter(site,payload):
    filtered_df2 = spacex_df[(spacex_df['Payload Mass (kg)'] > payload[0]) & (spacex_df['Payload Mass (kg)'] < payload[1])]
    
    if site == 'ALL':
        fig= px.scatter(filtered_df2, x="Payload Mass (kg)", y="class", color="Booster Version", title=f"Correlation between Payload and Success for All sites")
        return fig
    else:
        filtered_df2 = filtered_df2[filtered_df2['Launch Site'] == site]
        fig= px.scatter(filtered_df2, x= "Payload Mass (kg)", y="class",color="Booster Version",title=f"Correlation between Payload and Success for Site {site}")
        return fig
                                    
# Run the app
if __name__ == '__main__':
    #app.run_server()
# Run app and display result inline in the notebook
    app.run_server(mode='inline')
